In [1]:
! pip install -U torchtext==0.12 torchdata==0.3.0 spacy torch
! python -m spacy download en_core_web_sm
! python -m spacy download de_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.4 MB 27.9 MB/s 
     |████████████████████████████████| 47 kB 5.2 MB/s 
     |████████████████████████████████| 6.4 MB 59.3 MB/s 
     |██████████████████████████████  | 834.1 MB 1.2 MB/s eta 0:00:46tcmalloc: large alloc 1147494400 bytes == 0x3941e000 @  0x7f3014a9d615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████████████████████| 890.2 MB 7.1 kB/s 
     |████████████████████████████████| 750.6 MB 12 kB/s 
     |████████████████████████████████| 140 kB 68.2 MB/s 
     |████████████████████████████████| 127 kB 72.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    U

In [2]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List


# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# Place-holders
token_transform = {}
vocab_transform = {}


# Create source and target language tokenizer. Make sure to install the dependencies.
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

In [3]:
# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set UNK_IDX as the default index. This index is returned when the token is not found.
# If not set, it throws RuntimeError when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/datapipes/iter/combining.py:181: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  "the buffer and each child DataPipe will read from the start again.", UserWarning)


In [4]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [5]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [6]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [7]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# src and tgt language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tesors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [8]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader)) # FIXME there was an error there 

In [9]:
from timeit import default_timer as timer
NUM_EPOCHS = 18

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))



Epoch: 1, Train loss: 5.344, Val loss: 4.114, Epoch time = 42.386s
Epoch: 2, Train loss: 3.761, Val loss: 3.320, Epoch time = 43.677s
Epoch: 3, Train loss: 3.162, Val loss: 2.895, Epoch time = 45.961s
Epoch: 4, Train loss: 2.768, Val loss: 2.639, Epoch time = 46.378s
Epoch: 5, Train loss: 2.481, Val loss: 2.442, Epoch time = 46.655s
Epoch: 6, Train loss: 2.250, Val loss: 2.315, Epoch time = 46.308s
Epoch: 7, Train loss: 2.060, Val loss: 2.207, Epoch time = 46.434s
Epoch: 8, Train loss: 1.897, Val loss: 2.115, Epoch time = 46.395s
Epoch: 9, Train loss: 1.754, Val loss: 2.060, Epoch time = 46.439s
Epoch: 10, Train loss: 1.631, Val loss: 2.002, Epoch time = 46.310s
Epoch: 11, Train loss: 1.523, Val loss: 1.973, Epoch time = 47.170s
Epoch: 12, Train loss: 1.419, Val loss: 1.941, Epoch time = 46.473s
Epoch: 13, Train loss: 1.333, Val loss: 1.970, Epoch time = 46.563s
Epoch: 14, Train loss: 1.252, Val loss: 1.945, Epoch time = 46.429s
Epoch: 15, Train loss: 1.174, Val loss: 1.934, Epoch time

In [11]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)

        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        #decoder prediction
        prob = model.generator(out[:, -1])
        #greedy selection
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

In [83]:
# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [84]:
print(translate(transformer, "Der Hund läuft auf der Straße"))

 The dog is running on the street . 


## Decoding functions

#### 1. Top-K sampling
Top-K sampling where the K next words are filtered and the probability mass is redistributed among only those K next words. GPT2 adopted this sampling scheme, which was one of the reasons for its success in story generation.

In [71]:
def topk_decode(model, src, src_mask, max_len, start_symbol, k):
    """
    model - model
    max_len - max length of the output
    k - k number next words
    """
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)

        #decode in ys the tgt_mask
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])

        #softmax the probability to normalize TODO
        prob = torch.softmax(prob, dim=1)
        
        #topk selection
        k_prob, k_index = torch.topk(prob, k=k, dim=1)
        multi_index = torch.multinomial(k_prob, num_samples=1)
        
        #choosing next_word
        next_word = k_index[0][multi_index].item()
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)

        if next_word == EOS_IDX:
            break
    return ys

In [72]:
# actual function to translate input sentence into target language
def translate_topk(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = topk_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX, k=5).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

When testing on some sentence we get a pretty decent result :

In [73]:
print(translate_topk(transformer, "Ein Mädchen spielt nicht gern Klavier"))
print(translate_topk(transformer, "Der Hund läuft auf der Straße"))
print(translate_topk(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))

 A girl playing n't be does not not doing a piano .
 The dog is running on the street . 
 A group of people standing in front of an anvil . 


In [74]:
def topk_decode_temperature(model, src, src_mask, max_len, start_symbol, k, temperature):
    """
    model - model
    max_len - max length of the output
    k - k number next words
    temeprature - 
    """
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)

        #decode in ys the tgt_mask
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])

        #softmax the probability to normalize TODO
        prob = torch.softmax(prob, dim=1)
        
        #topk selection
        top_k_prob, top_k_index = torch.topk(prob / temperature, k=k, dim=1)
        multi_index = torch.multinomial(top_k_prob, num_samples=1)
        
        #choosing next_word
        next_word = top_k_index[0][multi_index].item()
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)

        if next_word == EOS_IDX:
            break
    return ys

In [75]:
# actual function to translate input sentence into target language
def translate_topk_temperature(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = topk_decode_temperature(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX, k=5, temperature=0.5).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [76]:
print(translate_topk_temperature(transformer, "Ein Mädchen spielt nicht gern Klavier"))
print(translate_topk_temperature(transformer, "Der Hund läuft auf der Straße"))
print(translate_topk_temperature(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))

 A girl plays music that is not performing a piano . 
 The dog is running on the street . 
 A group of people standing in front an anvil . 


#### 2. Beam search 

In [77]:
def create_branch(memory, ys, prob):
  return {
            "memory": memory,
            "ys": ys,
            "prob": prob
        }

In [82]:
def beam_decode(model, src, src_mask, max_len, start_symbol, k):
  src = src.to(DEVICE)
  src_mask = src_mask.to(DEVICE)

  memory = model.encode(src, src_mask)
  # ---------- init

  ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
  #first run
  memory = memory.to(DEVICE)
  tgt_mask = (generate_square_subsequent_mask(ys.size(0))
              .type(torch.bool)).to(DEVICE)

  #decode in ys the tgt_mask
  out = model.decode(ys, memory, tgt_mask)
  out = out.transpose(0, 1)
  prob = model.generator(out[:, -1])

  #softmax the probability to normalize TODO
  prob = torch.softmax(prob, dim=1)
  
  #topk selection
  k_prob, k_index = torch.topk(prob, k=k, dim=1)
  multi_index = torch.multinomial(k_prob, num_samples=1)
  score = torch.multinomial(k_prob, 1).item()
  
  #choosing next_word
  next_word = k_index[0][multi_index].item()
  ys = torch.cat([ys,
                  torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
  if next_word == EOS_IDX:
    return ys
  # ---------- init
  
  branch = [create_branch(memory, ys, score)] * k
  
  for i in range(max_len-1):
    new_branch = []
    for actual in branch:
      memory = actual["memory"]
      memory = memory.to(DEVICE)

      ys = actual["ys"]
      
      # ---------- 
      tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                  .type(torch.bool)).to(DEVICE)
      out = model.decode(ys, memory, tgt_mask)
      out = out.transpose(0, 1)
      prob = model.generator(out[:, -1])
      prob = torch.softmax(prob, dim=1)
      k_prob, k_index = torch.topk(prob, k=k, dim=1)
      # ----------

      ys = torch.cat([ys, 
                      torch.ones(1, 1).type_as(src.data).fill_(next_word)], 
                      dim=0)
      
      for j in range(k):
        next_word = k_index[0][j].item()
        new_branch.append(create_branch(memory, ys, actual["prob"] * torch.multinomial(k_prob, 1).item())
      new_branch = sorted(new_branch, key=lambda x: x["prob"], reverse=True)
      branch = new_branch[:k]
      if branch[0]["ys"][-1] == EOS_IDX:
        break
  return branch[0]["ys"]

SyntaxError: ignored

In [134]:
def beam_decode2(model, src, src_mask, max_len, start_symbol, k=5):
    """
      src: source sentence
      src_mask: source mask
      max_len: maximum length of output sentence
      k: beam size
    """
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)

    # simple topk to get a first prob
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    memory = memory.to(DEVICE)
    tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                .type(torch.bool)).to(DEVICE)
    out = model.decode(ys, memory, tgt_mask)
    out = out.transpose(0, 1)
    prob = model.generator(out[:, -1])
    prob = torch.softmax(prob, dim=1)
    k_prob, k_index = torch.topk(prob, k=k, dim=1)
    random_index = torch.multinomial(k_prob, 1)
        
    next_word = k_index[0][random_index].item()
    ys = torch.cat([ys,
                    torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
    if next_word == EOS_IDX:
        return ys

    # initialize the rest of the branch
    init_branch = {
            "memory": memory,
            "ys": ys,
            "score": torch.multinomial(k_prob, 1).item()
        }
    branch = [init_branch] * k

    for i in range(max_len-1):
        new_branch = []
        for elt in branch:
            memory = elt["memory"].to(DEVICE)
            ys = elt["ys"]
            
            tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                        .type(torch.bool)).to(DEVICE)
            out = model.decode(ys, memory, tgt_mask)
            out = out.transpose(0, 1)
            prob = model.generator(out[:, -1])
            prob = torch.softmax(prob, dim=1)
            k_prob, k_index = torch.topk(prob, k=k, dim=1)
            score = torch.multinomial(k_prob, 1).item()

            for j in range(k):
                next_word = k_index[0][j].item()
                new_branch.append({
                    "memory": memory,
                    "ys": torch.cat([ys,
                    torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0),
                    "score": elt["score"] * score})
        #sort the list of all new branch 
        new_branch = sorted(new_branch, key=lambda x: x["score"], reverse=True)
        branch = new_branch[:k]
        ys = branch[0]["ys"]

        if ys[-1] == EOS_IDX:
            break
        
    return ys

In [135]:
# actual function to translate input sentence into target language
def translate_beam(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = beam_decode2(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX, k=5).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [138]:
print(translate_beam(transformer, "Der Hund läuft auf der Straße"))

 The dog is running on the street . 


### 3. Comparison between the decoding functions

In [142]:
sentences = ["Ich schlafe gerade nicht viel",
             "Abends geht das Licht auf der Straße aus",
             "Schau in den Himmel und sieh"]
traduction = ["I'm not sleeping much right now",
              "In the evening the lights go out on the street",
              "Look up to the sky and see"]

In [1]:
for i in range(3):
  print("traduction ->", traduction[i])
  print("Greedy decoder -",translate(transformer, sentences[i]))
  print("Topk decoder -",translate_topk(transformer, sentences[i]))
  print("Topk temperature = 5 decoder -",translate_topk_temperature(transformer, sentences[i]))
  print("Beam decoder -",translate_beam(transformer, sentences[i]), end="\n\n")

NameError: ignored